### Data exploration

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('data/titanic/train.csv')
test = pd.read_csv('data/titanic/test.csv')

In [ ]:
# Class imbalance
import seaborn as sns
print(train['Survived'].value_counts())
print(train['Survived'].value_counts(normalize=True))

# 1. Class imbalance (Survival counts)
sns.countplot(data=train, x='Survived')
plt.title('Survived vs. Not Survived')
plt.show()

In [ ]:
# 2. Numerical features distributions
sns.histplot(train['Age'])
plt.title('Age Distribution')
plt.show()

sns.histplot(train['SibSp'])
plt.title('Siblings/Spouses Aboard')
plt.show()

sns.histplot(train['Parch'])
plt.title('Parents/Children Aboard')
plt.show()

sns.histplot(train['Fare'])
plt.title('Fare Distribution')
plt.show()

In [ ]:
# 3. Categorical features distributions
sns.countplot(data=train, x='Pclass')
plt.title('Passenger Class Distribution')
plt.show()

sns.countplot(data=train, x='Sex')
plt.title('Sex Distribution')
plt.show()

sns.countplot(data=train, x='Embarked')
plt.title('Port of Embarkation Distribution')
plt.show()

In [ ]:
# Survival rate by sex
sns.barplot(data=train, x='Sex', y='Survived')
plt.title('Survival Rate by Gender')

In [ ]:
# Survival rate by passenger class
sns.barplot(data=train, x='Pclass', y='Survived')
plt.title('Survival Rate by Passenger Class')

In [ ]:
# Survival rate by age
train['Age Group'] = pd.cut(train['Age'], bins=[0, 12, 18, 35, 60, 80], labels=['Child', 'Teen', 'Young Adult', 'Adult', 'Senior'])

sns.barplot(data=train, x='Age Group', y='Survived')
plt.title('Survival Rate by Age Group')

### Data Preprocessing

In [ ]:
# Combined training & testing data sets to handle missing values later

combined = pd.concat([train, test], ignore_index=True)

In [ ]:
# Missing value count per feature

test.isna().sum()

In [ ]:
# Extracting titles (Characters after the comma & before the dot)

combined['Title'] = combined['Name'].str.extract(r',\s*([^\.]+)\.', expand=False)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# Define different categoric features
categorical_features = ['Sex', 'Embarked', 'Cabin', 'Pclass', 'Title']

# Categorical transfomer pipeline

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
# Numeric_features
numeric_features = ['Age', 'Fare', 'SibSp', 'Parch']

# Numeric transfomer pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

In [ ]:
# Preprocessing steps 

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numeric_features)
    ]
)

In [ ]:
# Dropped the 'Age Group' column I added before to
# visualize the survival rate by age

combined.drop('Age Group', inplace=True, axis=1)

### Model Building

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [ ]:
# Split the combined data set

train_data = combined.iloc[:891].copy()
test_data = combined.iloc[891:].copy()

In [ ]:
X = train_data.drop(['Survived', 'Name', 'Ticket'], axis=1)
y = train_data['Survived']

In [ ]:
# Train-test split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
# Used Pipeline since some columns have non numerical values
# which couldn't be converted when I tried to fit my models

classification_models = {
        'Logistic Regression' : Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', LogisticRegression(max_iter=1000))
        ]),
        'RandomForestClassifier' : Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier())
        ]),
        'XGBoost' : Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', XGBClassifier())
        ])
    }

results = {}

In [ ]:
np.random.seed(42) # To avoid randomness

for mode_name, model in classification_models.items():
    model.fit(X_train, y_train)
    results[mode_name] = model.score(X_test, y_test)
results

In [ ]:
results_df = pd.DataFrame(results.values(),
                          results.keys(),
                          columns=['Accuracy'])
results_df.plot.bar()

# Notice that Logistic Regression scored the highest, with ~82% accuracy

### Logistic Regression Tuning

In [ ]:
# Logistic Regression hyperparameters
from sklearn.model_selection import GridSearchCV
log_reg_grid = {"classifier__C": np.logspace(-4, 4, 20),
                "classifier__solver": ["liblinear"]}

In [ ]:
# Hyperparameter Tuning using GridSearchCV

pipe = Pipeline([
    ('estimator', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

gs_log = GridSearchCV(estimator=pipe,
             param_grid=log_reg_grid)

gs_log.fit(X_train, y_train)

In [ ]:
# Finding best parameters for tuning

gs_log.best_params_

In [ ]:
gs_log.score(X_test, y_test)

### Model Evaluation


In [ ]:
clf = LogisticRegression(solver='liblinear', C=29.763514416313132)

pipe = Pipeline([
    ('estimator', preprocessor),
    ('classifier', clf)
])

pipe.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix


In [ ]:
y_preds = pipe.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_preds)

In [ ]:
def plot_conf_mat(y_test, y_preds):
    """
    Plots a confusion matrix using Seaborn's heatmap().
    """
    fig, ax = plt.subplots(figsize=(3, 3))
    ax = sns.heatmap(confusion_matrix(y_test, y_preds),
                     annot=True,
                     cbar=False)
    plt.xlabel("True label")
    plt.ylabel("Predicted label")
    
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5);
    
plot_conf_mat(y_test, y_preds)

In [ ]:
accuracy_score(y_test, y_preds)

In [ ]:
precision_score(y_test, y_preds)

In [ ]:
recall_score(y_test, y_preds)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
# Cross-validated Accuracy
cross_val_acc = np.mean(cross_val_score(pipe,
                                        X,
                                        y,
                                        scoring="accuracy",
                                        cv=5))

cross_val_acc

In [ ]:
# Cross-validated Precision
cross_val_precision = np.mean(cross_val_score(pipe,
                                              X,
                                              y,
                                              scoring="precision",
                                              cv=5))

cross_val_precision

In [ ]:
# Cross-validated Recall
cross_val_recall = np.mean(cross_val_score(pipe,
                                           X,
                                           y,
                                           scoring="recall",
                                           cv=5))

cross_val_recall

### Final Predictions

After preprocessing the data (handling missing values, encoding categorical variables, and extracting titles from names), we trained and evaluated several classification models:      Logistic Regression, Random Forest Classifier, XGBoost Classifier.

All three models achieved similar accuracy:

'Logistic Regression': 0.8212,
'RandomForestClassifier': 0.8045,
'XGBoost': 0.8045

After hyperparameter tuning, Logistic Regression actually performed worse, dropping from 0.82 to 0.79.

### Insights

Feature engineering, especially extracting titles from names (like Mr., Mrs., Miss), had a positive impact on all models.

Tuning isn't always beneficial. In this case, tuning Logistic Regression may have led to overfitting or pushed it into a suboptimal configuration.